In [74]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary
import collections
from itertools import product

# 1. 加载数据
file_path = "realdata_v1.xlsx"  # 请替换为你的文件路径
activities_df = pd.read_excel(file_path, sheet_name="ActivitiesInfo")
classroom_df = pd.read_excel(file_path, sheet_name="ClassroomsInfo")
student_courses_df = pd.read_excel(file_path, sheet_name="StudentsInfo")

### 模型一：

In [75]:
# Index set
courses = student_courses_df["Course_ID"]
courses = set([int(num) for row in courses for num in row.split(', ')])

# classes = student_courses_df["Class_ID"]
# classes = set([int(row) for row in classes])

classrooms = classroom_df["Classroom_ID"]
classrooms = set(int(row) for row in classrooms)

weeks = classroom_df["Available_Weeks"]
weeks = set([int(num) for row in weeks for num in row.split(', ')])

time_slots = classroom_df["Time_Slot"]
time_slots = set([int(num) for row in time_slots for num in row.split(', ')])

activities = activities_df["Activity_ID"]
activities = set([int(row) for row in activities])

activity_types = activities_df["Activity_Type"]
activity_types = set(activity_types)

In [76]:
from collections import Counter
for _, row in activities_df.iterrows():
    row_prefer_weeks = [int(i) for i in str(row["Preferred_Weeks"]).split(', ')]
    row_available_weeks = [int(i) for i in str(row["Available_Weeks"]).split(', ')]
    for week in row_prefer_weeks:
        if week not in row_available_weeks:
            print('1', row["Course_ID"], row["Activity_ID"], row["Activity_Type"], row["Num_Students"], week)
for _, row in activities_df.iterrows():
    row_prefer_weeks = [int(i) for i in str(row["Preferred_Weeks"]).split(', ')]
    row_available_weeks = [int(i) for i in str(row["Available_Weeks"]).split(', ')]
    row_check = Counter(row_available_weeks)
    for k, v in row_check.items():
        if v > 1:
            print('2', row["Course_ID"], row["Activity_ID"], row["Activity_Type"], row["Num_Students"], k)


In [77]:
# 参数
B = ["morning", "afternoon"]

# 预先设定好的参数（读表）
Gsa = [(row["Course_ID"], row["Activity_ID"]) for _, row in activities_df.iterrows() if row["Requires_Separation"] == 1]
Ysai = [(row["Course_ID"], row["Activity_ID"], row["Activity_Type"]) for _, row in activities_df.iterrows()]
Csa = [(row["Course_ID"], row["Activity_ID"]) for _, row in activities_df.iterrows() if row["Requires_Computers"] == 1]
Tsa = [(row["Course_ID"], row["Activity_ID"]) for _, row in activities_df.iterrows() if row["Requires_Tables"] == 1]
Bsab = [(row["Course_ID"], row["Activity_ID"], row["Preferred_Time"]) for _, row in activities_df.iterrows() if row["Preferred_Time"] in B]

Gamma_sa = [(row["Course_ID"], row["Activity_ID"]) for _, row in activities_df.iterrows() if row["Activity_Type"] in [2, 3]]
Ltb = [(0, "morning"), (1, "morning"), (2, "afternoon"), (3, "afternoon")]
Ltb_mapping = {row[0]: row[1] for row in Ltb}
Omega_tt = {}
for t1 in range(len(time_slots)):
    for t2 in range(len(time_slots)):
        if Ltb_mapping.get(t1, 0) == Ltb_mapping.get(t2, 0):
            Omega_tt[(t1, t2)] = 1
max_courses_per_week = 5
Psak = {(row["Course_ID"], row["Activity_ID"], k): int(week) 
        for _, row in activities_df.iterrows() 
        for k, week in enumerate(str(row["Preferred_Weeks"]).split(', '))}
Wk = {k: penalty for k, penalty in zip(range(3), [0, 0.1, 0.3])}

activity_type_mapping = {row["Activity_ID"]: row["Activity_Type"] for _, row in activities_df.iterrows()}

# 计算 O_ss': 需要避免冲突的课程对
# Oss = [(row["Course_ID"], row2["Course_ID"]) 

#        for _, row in activities_df.iterrows() 
#        for _, row2 in activities_df.iterrows()
#        if row["Course_ID"] != row2["Course_ID"] and row["Requires_Separation"] == 1 and row2["Requires_Separation"] == 1]

In [78]:
# 缩小维度，简化计算
available_weeks = {(row["Course_ID"], row["Activity_ID"]): str(row["Available_Weeks"]).split(', ') for _, row in activities_df.iterrows()}
available_weeks = {k: [int(vv) for vv in v] for k, v in available_weeks.items()}
sa = [(s, a) for s in courses for a in activities_df[activities_df["Course_ID"] == s]["Activity_ID"]]

# 3. 定义模型
model = LpProblem(name="course_scheduling", sense=LpMinimize)

# 变量
y = {(w, s, a): LpVariable(f"y_{w}_{s}_{a}", cat=LpBinary) for s, a in sa for w in available_weeks[(s, a)]}
x = {(w, t, s, a): LpVariable(f"x_{w}_{t}_{s}_{a}", cat=LpBinary) 
     for s, a in sa for w in available_weeks[(s, a)] for t in time_slots}

model += lpSum(Wk[k] * y[Psak[s, a, k], s, a] for k in range(1, 3) for s, a in sa if Psak.get((s, a, k), -1) != -1), "Minimize_Preference_Penalty"

# 4. A.2 - A.11)
# A.2
for s, a in sa:
    model += lpSum(x[w, t, s, a] for t in time_slots for w in available_weeks[(s, a)]) >= 1, f"Ensure_One_Assignment_{s}_{a}"

# A.3
for s, a in sa:
    for b in B:
        for w in available_weeks[(s, a)]:
            model += lpSum(
                x[w, t, s, a] for t in time_slots if (t, b) in Ltb
            ) <= 1, f"Max_One_Per_Limited_Slot_{s}_{a}_{b}_{w}"

# A.4
for s, a in sa:
    for b in B:
        if (s, a, b) not in Bsab:
            for w in available_weeks[(s, a)]:
                model += lpSum(x[w, t, s, a] for t in time_slots if (t, b) in Ltb) == 0, f"Forbidden_Resource_{s}_{a}_{b}_{w}"

# A.5
for s, a in sa:
    for t in time_slots:
        for t_prime in time_slots:
            for w in available_weeks[(s, a)]:
                if Omega_tt.get((t, t_prime), 0) == 0 or (s, a) not in Gsa:
                    model += x[w, t, s, a] + x[w, t_prime, s, a] <= y[w, s, a], f"Time_Slot_Limit_{w}_{s}_{a}_{t}_{t_prime}"

# A.6
for s, a in sa:
    for w in available_weeks[(s, a)]:
        for b in B:
            for b_prime in B:
                if b != b_prime or ((s, a, b) in B and (s, a, b_prime) in B):
                    model += (
                        lpSum(x[w, t, s, a] for t in time_slots if (t, b) in Ltb) +
                        lpSum(x[w, t_prime, s, a] for t_prime in time_slots if (t_prime, b_prime) in Ltb)
                    ) >= 2 * y[w, s, a], f"Resource_Conflict_{w}_{s}_{a}_{b}_{b_prime}"

# A.7
for w in weeks:
    model += lpSum(
        y[w, s, a] for s, a in sa if (s, a) in Gamma_sa and w in available_weeks[(s, a)]
    ) <= max_courses_per_week, f"Max_Courses_Per_Student_{w}"

# A.8
# for t in time_slots:
#     for s, a in sa:
#         for s_prime, a_prime in sa:
#             if s != s_prime and a != a_prime and Oss.get((s, s_prime), 0) == 1:
#                 for w in available_weeks[(s, a)]:
#                     model += x[w, t, s, a] + x[w, t, s_prime, a_prime] <= 1, f"Class_Conflict_{w}_{t}_{s}_{s_prime}_{a}_{a_prime}"

# A.9
for t in time_slots:
    for s, a in sa:
        for s_prime, a_prime in sa:
            if s != s_prime and a != a_prime:
                for w in available_weeks[(s, a)]:
                    if (s, a) in Gsa and (s_prime, a_prime) in Gsa and (s, a) in Tsa and (s_prime, a_prime) in Tsa and w in available_weeks[(s_prime, a_prime)]:
                        model += x[w, t, s, a] + x[w, t, s_prime, a_prime] <= 1, f"Group_Time_Conflict_{w}_{t}_{s}_{s_prime}_{a}_{a_prime}"

# 5. 求解
model.solve()

w_schedule_results = [
    {"Week": w, "Course_ID": s, "Activity_ID": a, "Probability": y[w, s, a].value()}
    for w, s, a in y if y[w, s, a].value() >= 0.5
]

w_t_schedule_results = [
    {"Week": w, "Time_slot": t, "Course_ID": s, "Activity_ID": a, "Probability": x[w, t, s, a].value()}
    for w, t, s, a in x if x[w, t, s, a].value() >= 0.5
]

w_schedule_df = pd.DataFrame(w_schedule_results)

w_t_schedule_df = pd.DataFrame(w_t_schedule_results)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/pp2/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/45077b116cb848e2a918eeb47f47118b-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/45077b116cb848e2a918eeb47f47118b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 43112 COLUMNS
At line 166522 RHS
At line 209630 BOUNDS
At line 213971 ENDATA
Problem MODEL has 43107 rows, 4340 columns and 114626 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.60 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.66   (Wallclock seconds):       0.70



In [79]:
w_schedule_df

,Week,Course_ID,Activity_ID,Probability
0,10,1,102,0.5
1,10,2,201,0.5
2,12,2,202,0.5
3,13,4,401,0.5
4,11,6,602,0.5
5,6,7,701,0.5
6,10,7,702,0.5
7,14,7,703,0.5
8,18,7,704,0.5
9,14,9,902,0.5


In [80]:
w_t_schedule_df

,Week,Time_slot,Course_ID,Activity_ID,Probability
0,10,2,1,102,0.5
1,10,3,1,102,0.5
2,10,2,2,201,0.5
3,10,3,2,201,0.5
4,12,2,2,202,0.5
...,...,...,...,...,...
59,36,3,50,5001,0.5
60,27,2,51,5101,0.5
61,27,3,51,5101,0.5
62,29,2,53,5301,0.5


##### 结果处理
1. **按照(week, time slot)归类活动候选集合**：例如week 1 time slot 1，候选集合：{"course1", "course3", ...}。（题目假设每周每个学科只能举办一个活动）

2. **去除线上考试的活动**

3. **结果进行备份**：csv格式如下
|week|time_slot|subject_candidates|

4. **对于每周每个时间点，运行一次模型二**

In [81]:
def process_data(df):
    df = df[~df["Activity_ID"].map(activity_type_mapping).eq(2)]
    grouped = df.groupby(["Week", "Time_slot"])["Course_ID"].apply(lambda x: set(x)).reset_index()
    grouped.rename(columns={"Course_ID": "subject_candidates"}, inplace=True)
    grouped.to_csv("processed_schedule.csv", index=False)
    
    return grouped
subject_candidates = process_data(w_t_schedule_df)
subject_candidates

,Week,Time_slot,subject_candidates
0,6,2,{7}
1,6,3,{7}
2,10,2,"{2, 7, 15, 16, 21, 24}"
3,10,3,"{2, 7, 15, 16, 21, 24}"
4,11,2,{28}
5,11,3,{28}
6,12,2,{2}
7,12,3,{2}
8,13,2,{26}
9,13,3,{26}


### 模型二：

In [82]:
# 模型二的很多参数是由模型一的参数缩小维度而来，比如模型一有一个参数：Ysai，模型二有一个参数Ysi，去掉了a这个维度
# 参数
Pc = [row["Classroom_ID"] for _, row in classroom_df.iterrows() if row["Has_Computers"] == 1]
Ysi = {(row["Course_ID"], row["Activity_Type"]): 1 for _, row in activities_df.iterrows()}
Gs = {row["Course_ID"]: 1 for _, row in activities_df.iterrows() if row["Requires_Separation"] == 1}
Cs = {row["Course_ID"]: 1 for _, row in activities_df.iterrows() if row["Requires_Computers"] == 1}
Ts = {row["Course_ID"]: 1 for _, row in activities_df.iterrows() if row["Requires_Tables"] == 1}
Tc = {row["Classroom_ID"]: 1 for _, row in classroom_df.iterrows() if row["Has_Tables"] == 1}
Act = {(row["Classroom_ID"], int(w), int(t)): 1 for _, row in classroom_df.iterrows() for w in row["Available_Weeks"].split(', ') for t in row["Time_Slot"].split(', ')} #
Ic = {row["Classroom_ID"]: 1 for _, row in classroom_df.iterrows() if row["Is_Isolated"] == 1}
Ns = {row["Course_ID"]: row["Num_Students"] for _, row in activities_df.iterrows()}
Msg = collections.defaultdict(int)
for course in courses:
    for _, row in student_courses_df.iterrows():
        if str(course) in row["Course_ID"]:
            Msg[(course, row["Class_ID"])] += 1

class_courses = collections.defaultdict(set)
for (course, class_id), _ in Msg.items():
    class_courses[course].add(class_id)

Capacity = {row["Classroom_ID"]: row["Capacity"] for _, row in classroom_df.iterrows()}
Occupancy_rate = {1: 0.9, 2: 0, 3: 1}
CAPci = {(c, i): int(Capacity[c] * Occupancy_rate[i]) for c, i in product(classrooms, activity_types)}

KeyError: 'Class_ID'

In [ ]:
df = pd.read_csv("processed_schedule.csv")
count = 0
week = df.iloc[count]["Week"]
time_slot = df.iloc[count]["Time_slot"]
subjects = df["subject_candidates"].tolist()
subjects = [int(i) for i in subjects[count].split(', ')]

In [85]:
# 2. 定义优化模型
model = LpProblem(name="classroom_assignment", sense=LpMinimize)

z = {(c, s): LpVariable(f"z_{c}_{s}", cat=LpBinary) for c in classrooms for s in subjects}
w = {(c, s, g): LpVariable(f"w_{c}_{s}_{g}", cat=LpBinary) for c in classrooms for s in subjects for g in class_courses[s]}

# 目标函数
model += lpSum(z[c, s] for c in classrooms for s in subjects), "Minimize_Classroom_Usage"

# A.13 - 确保教室容量足够
for s in subjects:
    model += lpSum(z[c, s] * sum(CAPci.get((c, i), 0) * Ysi.get((s, i), 0) for i in activity_types) for c in classrooms) >= Ns[s], f"Capacity_Constraint_{s}"

# A.14 - 确保每个班级的教室容量足够
for s in subjects:
    if Ts.get(s, 0) == 1:
        for g in class_courses[s]:
            model += lpSum(w[c, s, g] * sum(CAPci.get((c, i), 0) * Ysi.get((s, i), 0) for i in activity_types) for c in classrooms) >= Msg.get((s, g), 0), f"Classroom_Capacity_{s}_{g}"

# A.15 - 教室不可重复使用
for c in classrooms:
    model += lpSum(z[c, s] for s in subjects) <= 1, f"Single_Assignment_{c}"

# A.16 - 只允许可用的教室分配
for c in classrooms:
    for s in subjects:
        model += z[c, s] <= Act.get((c, week, time_slot), 0), f"Classroom_Availability_{c}_{s}"

# A.17 - 课时分配一致性
for c in classrooms:
    for s in subjects:
        if Gs.get(s, 0) == 1:
            for g in class_courses[s]:
                model += z[c, s] >= w[c, s, g], f"Consistency_1_{c}_{s}_{g}"

# A.18 - 课时分配一致性
for c in classrooms:
    for s in subjects:
        if Gs.get(s, 0) == 1:
            model += z[c, s] <= lpSum(w[c, s, g] for g in class_courses[s]), f"Consistency_2_{c}_{s}"

# A.19 - 确保不同班级不共享教室
for c in classrooms:
    for s in subjects:
        if Gs.get(s, 0) == 1:
            for g1 in class_courses[s]:
                for g2 in class_courses[s]:
                    if g1 != g2:
                        model += w[c, s, g1] + w[c, s, g2] <= 1, f"No_Shared_Classroom_{c}_{s}_{g1}_{g2}"

# A.20 - 计算机房需求
for c in classrooms:
    for s in subjects:
        if Cs.get(s, 0) == 1 and c not in Pc:
            model += z[c, s] == 0, f"Computer_Requirement_{c}_{s}"

# A.21 - 桌子需求
for c in classrooms:
    for s in subjects:
        if Ts.get(s, 0) == 1 and Tc.get(c, 0) == 0:  # 课程需要桌子但教室没有
            model += z[c, s] == 0, f"Table_Requirement_{c}_{s}"


# A.22 - 隔离教室
for c in classrooms:
    for s in subjects:
        if Ic.get(c, 0) == 1:
            model += lpSum(z[c_prime, s] for c_prime in classrooms if c_prime != c) <= (1 - z[c, s]) * len(classrooms), f"Isolation_Requirement_{c}_{s}"

# 3. 求解
model.solve()

classroom_results = [
    {"Classroom": c, "Course_ID": s}
    for c, s in z if z[c, s].value() >= 0.5
]

classroom_df = pd.DataFrame(classroom_results)
classroom_df

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/pp2/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/44b4ef6ef901449b863bd00a821edef7-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/44b4ef6ef901449b863bd00a821edef7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 361 COLUMNS
At line 2389 RHS
At line 2746 BOUNDS
At line 2947 ENDATA
Problem MODEL has 356 rows, 200 columns and 1567 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.575789 - 0.00 seconds
Cgl0002I 42 variables fixed
Cgl0003I 12 fixed, 0 tightened bounds, 30 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitu

,Classroom,Course_ID
0,103,1
1,107,6
2,109,6
3,112,5
4,116,6


In [ ]:
results = []
for count in range(len(subject_candidates)):
    week = df.iloc[count]["Week"]
    time_slot = df.iloc[count]["Time_slot"]
    subjects = df["subject_candidates"].tolist()
    subjects = [int(i) for i in subjects[count].split(', ')]

    # 2. 定义优化模型
    model = LpProblem(name="classroom_assignment", sense=LpMinimize)

    z = {(c, s): LpVariable(f"z_{c}_{s}", cat=LpBinary) for c in classrooms for s in subjects}
    w = {(c, s, g): LpVariable(f"w_{c}_{s}_{g}", cat=LpBinary) for c in classrooms for s in subjects for g in class_courses[s]}

    # 目标函数
    model += lpSum(z[c, s] for c in classrooms for s in subjects), "Minimize_Classroom_Usage"

    # A.13 - 确保教室容量足够
    for s in subjects:
        model += lpSum(z[c, s] * sum(CAPci.get((c, i), 0) * Ysi.get((s, i), 0) for i in activity_types) for c in classrooms) >= Ns[s], f"Capacity_Constraint_{s}"

    # A.14 - 确保每个班级的教室容量足够
    for s in subjects:
        if Ts.get(s, 0) == 1:
            for g in class_courses[s]:
                model += lpSum(w[c, s, g] * sum(CAPci.get((c, i), 0) * Ysi.get((s, i), 0) for i in activity_types) for c in classrooms) >= Msg.get((s, g), 0), f"Classroom_Capacity_{s}_{g}"

    # A.15 - 教室不可重复使用
    for c in classrooms:
        model += lpSum(z[c, s] for s in subjects) <= 1, f"Single_Assignment_{c}"

    # A.16 - 只允许可用的教室分配
    for c in classrooms:
        for s in subjects:
            model += z[c, s] <= Act.get((c, week, time_slot), 0), f"Classroom_Availability_{c}_{s}"

    # A.17 - 课时分配一致性
    for c in classrooms:
        for s in subjects:
            if Gs.get(s, 0) == 1:
                for g in class_courses[s]:
                    model += z[c, s] >= w[c, s, g], f"Consistency_1_{c}_{s}_{g}"

    # A.18 - 课时分配一致性
    for c in classrooms:
        for s in subjects:
            if Gs.get(s, 0) == 1:
                model += z[c, s] <= lpSum(w[c, s, g] for g in class_courses[s]), f"Consistency_2_{c}_{s}"

    # A.19 - 确保不同班级不共享教室
    for c in classrooms:
        for s in subjects:
            if Gs.get(s, 0) == 1:
                for g1 in class_courses[s]:
                    for g2 in class_courses[s]:
                        if g1 != g2:
                            model += w[c, s, g1] + w[c, s, g2] <= 1, f"No_Shared_Classroom_{c}_{s}_{g1}_{g2}"

    # A.20 - 计算机房需求
    for c in classrooms:
        for s in subjects:
            if Cs.get(s, 0) == 1 and c not in Pc:
                model += z[c, s] == 0, f"Computer_Requirement_{c}_{s}"

    # A.21 - 桌子需求
    for c in classrooms:
        for s in subjects:
            if Ts.get(s, 0) == 1 and Tc.get(c, 0) == 0:  # 课程需要桌子但教室没有
                model += z[c, s] == 0, f"Table_Requirement_{c}_{s}"


    # A.22 - 隔离教室
    for c in classrooms:
        for s in subjects:
            if Ic.get(c, 0) == 1:
                model += lpSum(z[c_prime, s] for c_prime in classrooms if c_prime != c) <= (1 - z[c, s]) * len(classrooms), f"Isolation_Requirement_{c}_{s}"

    # 3. 求解
    model.solve()

    classroom_results = [
        {"Classroom": c, "Course_ID": s}
        for c, s in z if z[c, s].value() >= 0.5
    ]

    classroom_df = pd.DataFrame(classroom_results)
    results.append(classroom_df)
    classroom_df

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/pp2/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/f7f9fed4746b4a359ad6a9fb39af82c9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/f7f9fed4746b4a359ad6a9fb39af82c9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 361 COLUMNS
At line 2389 RHS
At line 2746 BOUNDS
At line 2947 ENDATA
Problem MODEL has 356 rows, 200 columns and 1567 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.575789 - 0.00 seconds
Cgl0002I 42 variables fixed
Cgl0003I 12 fixed, 0 tightened bounds, 30 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitu

In [89]:
results

[   Classroom  Course_ID
 0        103          1
 1        107          6
 2        109          6
 3        112          5
 4        116          6,
    Classroom  Course_ID
 0        103          1
 1        107          6
 2        109          6
 3        112          5
 4        116          6,
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
    Classroom  Course_ID
 0        101          9
 1        109          3,
    Classroom  Course_ID
 0        101          9
 1        109          3,
    Classroom  Course_ID
 0        101          9
 1        109          3,
    Classroom  Course_ID
 0        101          9
 1        112          3,
    Classroom  Course_ID
 0        101          5
 1        110          7,
    Classroom  Course_ID
 0        101          5
 1        110          7,
 Empty DataFrame
 Columns: []
 Index: [],
    Classroom  Course_ID
 0     